## **PART 1: MAKING SPOTIFY API CALL**

**IMPORT LIBRARY**

In [1]:
#import libraries
import requests
import pandas as pd
import time

**TOKEN ID**

In [2]:
token = 'BQATzXxqWqR9JBL2gOwOstG6fJ3RLsS2fdhOSZtgcLNQBmlMjEEZGEitnXKcGJk0-3cZ_z8f1P-M5Zi85NPGzZGCBzeN33p-7tEq2P-KooX9mYEyL095OEkbl1tvKZFn3_NrPtYlpr_KxnKqqHZ1JYyJ0tmXg10'

**ARTISTS**

In [3]:
artists = ['drake','young thug','adele','taylor swift','kanye west','mac miller','juice wrld','doja cat','lil baby','the weeknd', 'morgan wallen','j. cole','rod wave','billie eilish', 'dababy', 'pop smoke', 'future', 'eminem', 'bad bunny', 'rod wave', 'billie eilish', 'ariana grande', 'luke combs', 'roddy ricch', 'travis scott', 'Xxxtentacion', 'justin bieber', 'gunna', 'lil wayne']

In [4]:
len(artists)

29

In [ ]:
artist_list = ['drake','young thug','youngboy never broke again',
               'adele','taylor swift','kanye west','mac miller',
               'juice wrld','doja cat','lil baby','the weeknd',
               'morgan wallen','j. cole','rod wave','billie eilish']

**API CALL CLASS**

In [5]:
class Artist_breakdown():

  def __init__(self, artist_name, token):
    self.artist_name = artist_name
    self.token = token
    self.artist_json_file = self.artist_search()
    self.genre = self.artist_json_info()[0]
    self.artist_id = self.artist_json_info()[1]
    self.artist_name = self.artist_json_info()[2]
    self.artist_popularity = self.artist_json_info()[3]

    self.album_json_file = self.album_search()
    self.album_type = self.album_info()[0]
    self.total_tracks = self.album_info()[1]
    self.release_date = self.album_info()[2]
    self.album_id = self.album_info()[3]
    self.available_markets = self.album_info()[4]
    self.album_name = self.album_info()[5]

    #self.track_json_file = self.track_search() ####
    self.duration_ms = [] #self.track_info()[1]
    self.track_id = [] #self.track_info()[2]
    self.track_title = self.track_info()[0]

    #self.track_deep_json_file = self.track_deep_search() ####
    self.track_popularity = self.track_deep_info()

  def artist_search(self):  
    token_url = 'https://api.spotify.com/v1/search'
    query = {'q': self.artist_name,'type':'artist'}
    authorization = {'Authorization': f'Bearer {self.token}'}
    artist_response = requests.get(token_url, params = query, headers = authorization).json()
    return artist_response

  def artist_json_info(self):
    artist_search_response = self.artist_json_file
    genre = artist_search_response['artists']['items'][0]['genres'][0]
    artist_id = artist_search_response['artists']['items'][0]['id']
    artist_name = artist_search_response['artists']['items'][0]['name']
    artist_popularity = artist_search_response['artists']['items'][0]['popularity']
    return genre,artist_id,artist_name,artist_popularity

  def album_search(self):
    token_url = f'https://api.spotify.com/v1/artists/{self.artist_id}/albums'
    query = {'id':self.artist_id}
    authorization = {'Authorization': f'Bearer {self.token}'}
    return requests.get(token_url, headers = authorization).json()

  #create function that will loop through the json object and retrieve Album ID, Album name, Release date, Total Tracks, Type
  def album_info(self):
    album_search_response = self.album_json_file

    album_type = [album_search_response['items'][0]['type']]
    total_tracks = [album_search_response['items'][0]['total_tracks']]
    release_date = [album_search_response['items'][0]['release_date']]
    album_id = [album_search_response['items'][1]['id']]
    available_markets = [len(album_search_response['items'][0]['available_markets'])]
    album_name = [album_search_response['items'][0]['name']]

    for item in album_search_response['items']:
      if item['name'] != album_name[-1]:
        album_type.append(item['type'])
        total_tracks.append(item['total_tracks'])
        release_date.append(item['release_date'])
        album_id.append(item['id'])
        available_markets.append(len(item['available_markets']))
        album_name.append(item['name'])
    return album_type, total_tracks, release_date, album_id, available_markets, album_name


  def track_search(self, album_id):
    token_url = f'https://api.spotify.com/v1/albums/{album_id}/tracks'
    authorization = {'Authorization': f'Bearer {self.token}'}
    return requests.get(token_url, headers = authorization).json()

  def track_info(self):
    track_title,duration_ms,track_id = [],[],[]
    genre, artist_id, artist_name, artist_popularity = [],[],[],[]
    album_type, total_tracks, release_date, album_id, available_markets, album_name = [],[],[],[],[],[]
    i = 0

    for id in self.album_id:
      response = self.track_search(id)
      time.sleep(0.2)
      for item in response['items']:
        track_title.append(item['name'])
        duration_ms.append(item['duration_ms'])
        track_id.append(item['id'])

        album_type.append(self.album_type[i])
        total_tracks.append(self.total_tracks[i])
        release_date.append(self.release_date[i])
        album_id.append(self.album_id[i])
        available_markets.append(self.available_markets[i])
        album_name.append(self.album_name[i])

      i += 1
    self.track_id = track_id
    self.album_type = album_type
    self.total_tracks = total_tracks
    self.release_date = release_date
    self.album_id = album_id
    self.available_markets = available_markets
    self.album_name = album_name

    self.genre = [self.genre] * len(self.track_id)
    self.artist_id = [self.artist_id] * len(self.track_id)
    self.artist_name = [self.artist_name] * len(self.track_id)
    self.artist_popularity = [self.artist_popularity] * len(self.track_id)

    self.track_title = track_title
    self.duration_ms = duration_ms
    self.track_id = track_id
    return track_title, duration_ms, track_id


  def track_deep_search(self, track_id):
    token_url = f'https://api.spotify.com/v1/tracks/{track_id}'
    authorization = {'Authorization': f'Bearer {self.token}'}
    return requests.get(token_url, headers = authorization).json()

  def track_deep_info(self):
    popularity = []
    for id in self.track_id:
      response = self.track_deep_search(id)
      popularity.append(response['popularity'])
      time.sleep(0.2)
    self.track_popularity = popularity
    return popularity


In [6]:
a,b,n,c,d,e,f,g,h,i,j,k,l,m = [],[],[],[],[],[],[],[],[],[],[],[],[],[]

In [7]:
for artist in artists:
  artist_info = Artist_breakdown(artist, token)
  a += artist_info.track_title
  b += artist_info.duration_ms
  n += artist_info.track_id
  c += artist_info.artist_name
  d += artist_info.artist_id
  e += artist_info.genre
  f += artist_info.artist_popularity
  g += artist_info.album_name
  h += artist_info.release_date
  i += artist_info.album_id
  j += artist_info.total_tracks
  k += artist_info.available_markets
  l += artist_info.album_type
  m += artist_info.track_popularity
my_dictionary = {'Track Title':a, 'Track Duration (sec)':b, 'Track ID': n,
                 'Artist':c, 'Artist ID':d, 'Genre':e,
                 'Artist Popularity':f, 'Album Name':g,
                 'Release Date':h,'Album ID':i, 'Total Tracks':j,
                 'Available Markets':k,'Album Type':l,'Track Popularity':m}

**DATAFRAME**

In [8]:
df = pd.DataFrame(my_dictionary)
df

,Track Title,Track Duration (sec),Track ID,Artist,Artist ID,Genre,Artist Popularity,Album Name,Release Date,Album ID,Total Tracks,Available Markets,Album Type,Track Popularity
0,Champagne Poetry,336511,7LSINVHqMXPt4FkTgcH6bt,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,100,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,178,album,50
1,Papi’s Home,178623,6C62fl8x0vzwxPqay8twie,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,100,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,178,album,45
2,Girls Want Girls (with Lil Baby),221979,60kRxS2TC4qb2WBPmRmfv4,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,100,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,178,album,55
3,In The Bible (with Lil Durk & Giveon),296568,7xXVDIulqmlBvDLFMfc3uX,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,100,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,178,album,45
4,Love All (with JAY-Z),228461,7L8V5vHLo6fMbNaVI7rbhU,Drake,3TVXtAsR1Inumwj472S9r4,canadian hip hop,100,Certified Lover Boy,2021-09-03,6sp02aeyiwfX35xRqwNiPv,21,178,album,45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4092,Trippy,263546,4SBC7RFDL8i1QBwMCfIiKX,Lil Wayne,55Aa2cqylxrFIXC767Z865,hip hop,90,I Am Not A Human Being II,2013-03-22,2EZ3h6cM5eC65qSHCghdIZ,14,151,album,17
4093,Love Me,255053,5LL5mK7OdCZ2FEldMOExNT,Lil Wayne,55Aa2cqylxrFIXC767Z865,hip hop,90,I Am Not A Human Being II,2013-03-22,2EZ3h6cM5eC65qSHCghdIZ,14,151,album,34
4094,God Bless Amerika,303000,4wLFAxDMU8Eb1cuumqBe1z,Lil Wayne,55Aa2cqylxrFIXC767Z865,hip hop,90,I Am Not A Human Being II,2013-03-22,2EZ3h6cM5eC65qSHCghdIZ,14,151,album,24
4095,Wowzerz,225360,2Rs2jS7FVMYiDnh9bqkzj6,Lil Wayne,55Aa2cqylxrFIXC767Z865,hip hop,90,I Am Not A Human Being II,2013-03-22,2EZ3h6cM5eC65qSHCghdIZ,14,151,album,14


In [9]:
df.describe()

,Track Duration (sec),Artist Popularity,Total Tracks,Available Markets,Track Popularity
count,4097.000000,4097.000000,4097.000000,4097.000000,4097.000000
mean,206088.758604,91.184525,17.725165,158.264096,47.253600
std,61748.604870,4.781212,7.537221,51.058240,20.250757
min,13578.000000,79.000000,1.000000,1.000000,0.000000
25%,171891.000000,89.000000,13.000000,174.000000,35.000000
50%,201615.000000,92.000000,17.000000,178.000000,50.000000
75%,236413.000000,95.000000,22.000000,178.000000,63.000000
max,875306.000000,100.000000,53.000000,178.000000,99.000000


## **PART 2: STORING PANDAS DATAFRAME ON POSTGRESQL DATABASE**

In [10]:
import psycopg2 as ps
import pandas as pd

**CONNECT TO DATABASE**

In [11]:
host_name = 'localhost'
dbname = 'spotify_tracks'
port = '5432'
username = 'postgres' 
password = 'password'
conn = None

In [12]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)

    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

In [13]:
conn = connect_to_db(host_name, dbname,port, username, password)

Connected!


In [14]:
curr = conn.cursor()

**CREATE TABLE**

In [15]:
def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS tracks (
                    track_id VARCHAR(255) PRIMARY KEY,
                    track_title VARCHAR(255) NOT NULL,
                    track_duration_sec NUMERIC NOT NULL,
                    artist VARCHAR(255) NOT NULL,
                    artist_id VARCHAR(255) NOT NULL,
                    genre VARCHAR(255) NOT NULL,
                    artist_popularity INTEGER NOT NULL,
                    album_name VARCHAR(255) NOT NULL,
                    release_date VARCHAR(50) NOT NULL,
                    album_id VARCHAR(255) NOT NULL,
                    total_tracks INTEGER NOT NULL,
                    available_markets INTEGER NOT NULL,
                    album_type VARCHAR(255) NOT NULL,
                    track_popularity INTEGER NOT NULL
            )""")
    curr.execute(create_table_command)

In [16]:
create_table(curr)

In [17]:
conn.commit()

**CHECK IF TRACK EXISTS ON DB**

In [18]:
def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['Track Title', 'Track Duration (sec)', 'Track ID', 'Artist', 'Artist ID', 
                                   'Genre', 'Artist Popularity', 'Album Name', 'Release Date', 'Album ID', 
                                   'Total Tracks', 'Available Markets', 'Album Type','Track Popularity'])
    for i, row in df.iterrows():
        if check_if_track_exists_on_db(curr, row['Track ID']): # If track already exists then we will update
            pass
            #update_row(curr,row['Track Title'],row['Track Duration (sec)'],row['Artist'],row['Artist ID'],row['Genre'],row['Artist Popularity'],row['Album Name'],row['Release Date'],row['Album ID'],row['Total Tracks'],row['Available Markets'],row['Album Type'],row['Track Popularity'])
        else: # The track doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)

    return tmp_df

In [19]:
def check_if_track_exists_on_db(curr, Track_id): 
    query = ("""SELECT track_id FROM tracks WHERE track_id = %s""")

    curr.execute(query, (Track_id,))
    return curr.fetchone() is not None

In [20]:
def update_row(curr, track_title, track_duration_sec, artist, artist_id, genre,artist_popularity, 
               album_name, release_date, album_id, total_tracks, available_markets, 
               album_type, track_popularity):
    query = ("""UPDATE videos
            SET Track_Title = %s,
                Track_Duration_sec = %s,
                Artist = %s,
                Artist_ID = %s,
                Genre = %s,
                Artist_Popularity = %s,
                Album_Name = %s,
                Release_Date = %s,
                Album_ID = %s,
                Total_Tracks = %s,
                Available_Markets = %s,
                Album_Type = %s,
                Track_Popularity = %s
            WHERE video_id = %s;""")
    vars_to_update = (track_title, track_duration_sec, artist, artist_id, genre,artist_popularity,
                      album_name, release_date, album_id, total_tracks, available_markets, 
                      album_type, track_popularity)
    curr.execute(query, vars_to_update)

In [21]:
new_tracks_df = update_db(curr,df)
conn.commit()

**INSERT DATA INTO DATAFRAME**

In [22]:
def append_from_df_to_db(curr,df):
    for i, row in df.iterrows():
        insert_into_table(curr,row['Track ID'],row['Track Title'],row['Track Duration (sec)'],
                          row['Artist'],row['Artist ID'],row['Genre'],row['Artist Popularity'],
                          row['Album Name'],row['Release Date'],row['Album ID'],row['Total Tracks'],
                          row['Available Markets'],row['Album Type'],row['Track Popularity'])

In [23]:
def insert_into_table(curr, track_id, track_title, track_duration_sec, artist, artist_id, genre, 
                      artist_popularity, album_name, release_date, album_id, total_tracks, 
                      available_markets, album_type, track_popularity):
    insert_into_tracks = ("""INSERT INTO tracks (track_id, track_title,track_duration_sec,
    artist,artist_id,genre, artist_popularity,album_name,release_date,album_id,total_tracks,
    available_markets,album_type,track_popularity)
    VALUES(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)
    ON CONFLICT (track_id)
    DO NOTHING;""")
    row_to_insert = (track_id, track_title, track_duration_sec, artist, artist_id, 
                     genre, artist_popularity, album_name, release_date, album_id, 
                     total_tracks, available_markets, album_type, track_popularity)
    curr.execute(insert_into_tracks, row_to_insert)

In [24]:
append_from_df_to_db(curr, new_tracks_df)
conn.commit()